In [2]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib as plt

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
IMAGE_SIZE = 256
BATCH_SIZE = 32

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset",
    shuffle=True,
    image_size=(IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

In [ ]:
class_names = dataset.class_names
class_names

In [ ]:
len(dataset)

In [ ]:
for image_batch, label_batch in dataset.take(1):
    print(image_batch.shape)
    print(label_batch.numpy())